In [29]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_distances


In [30]:
df = pd.read_csv('/Users/mac/Desktop/Mine/Data Science/Purwadhika/Lesson/Modul3/Session 23 - Recommendation System/exercise/dataset/imdb_top_1000.csv')
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


## Question
- Create new metadata with Genre, Overview, Director and Star 1-4
- Preprocessing the text with preprocessing part 1-3
- Create recommendation system with cosine distance
- Try the recommendation system is it good or bad ?

In [31]:
import warnings
warnings.filterwarnings('ignore')

## Combine into metadata

In [32]:
def combine_metadata(row):
    return f"{row['Genre']} {row['Overview']} {row['Director']} {row['Star1']} {row['Star2']} {row['Star3']} {row['Star4']}"

In [33]:
df['metadata'] = df.apply(combine_metadata, axis=1)
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,metadata
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Drama Two imprisoned men bond over a number of...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Crime, Drama An organized crime dynasty's agin..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Action, Crime, Drama When the menace known as ..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Crime, Drama The early life and career of Vito..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Crime, Drama A jury holdout attempts to preven..."


In [34]:
idx = 0

content = df.loc[idx, 'metadata']
content

'Drama Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. Frank Darabont Tim Robbins Morgan Freeman Bob Gunton William Sadler'

## Preprocessing 1

In [35]:
contractions_dict = {    
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i had",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "iit will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [36]:
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer 

stop_words = stopwords.words('english')
stop_words.remove('not')
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def remove_br_tags(text):
    cleaned_text = text.replace('<br />', '')
    return cleaned_text

def to_lower(text):
    return text.lower()

def remove_contraction(text):
    list_kata = text.split()
    list_hasil = []

    for kata in list_kata:

        if kata in contractions_dict.keys():
            list_hasil.append(contractions_dict[kata])
        else:
            list_hasil.append(kata)

    hasil = ' '.join(list_hasil)
    return hasil 

def remove_number(text):
    hasil = ''.join([char for char in text if not char.isnumeric()])
    return hasil

def remove_punctuation(text):
    hasil = ''.join([char for char in text if not char in punctuation])
    return hasil

def remove_stopwords(text):
    hasil = ' '.join([kata for kata in text.split() if kata not in stop_words])
    return hasil 

def remove_whitespace(text):
    hasil = ' '.join(text.split())
    return hasil

def stem(text):
    list_hasil = []

    for sentence in nltk.sent_tokenize(text):   # pecah paragraf menjadi kalimat-kalimat
        for word in nltk.word_tokenize(sentence): # pecah kalimat menjadi kata-kata
            list_hasil.append(stemmer.stem(word))   # ubah kata ke kata dasarnya

    hasil = ' '.join(list_hasil)
    return hasil 

def lemmatize(text):
    list_hasil = []

    for sentence in nltk.sent_tokenize(text):   # pecah paragraf menjadi kalimat-kalimat
        for word in nltk.word_tokenize(sentence): # pecah kalimat menjadi kata-kata
            list_hasil.append(lemmatizer.lemmatize(word))   # ubah kata ke kata dasarnya

    hasil = ' '.join(list_hasil)
    return hasil 

df['text-prep'] = df['metadata'].apply(remove_br_tags)
df['text-prep'] = df['text-prep'].apply(to_lower)
df['text-prep'] = df['text-prep'].apply(remove_contraction)
df['text-prep'] = df['text-prep'].apply(remove_number)
df['text-prep'] = df['text-prep'].apply(remove_punctuation)
df['text-prep'] = df['text-prep'].apply(remove_stopwords)
df['text-prep'] = df['text-prep'].apply(remove_whitespace)
df['text-prep-stem'] = df['text-prep'].apply(stem)
df['text-prep-lemm'] = df['text-prep'].apply(lemmatize)
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,metadata,text-prep,text-prep-stem,text-prep-lemm
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Drama Two imprisoned men bond over a number of...,drama two imprisoned men bond number years fin...,drama two imprison men bond number year find s...,drama two imprisoned men bond number year find...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Crime, Drama An organized crime dynasty's agin...",crime drama organized crime dynastys aging pat...,crime drama organ crime dynasti age patriarch ...,crime drama organized crime dynasty aging patr...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Action, Crime, Drama When the menace known as ...",action crime drama menace known joker wreaks h...,action crime drama menac known joker wreak hav...,action crime drama menace known joker wreaks h...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Crime, Drama The early life and career of Vito...",crime drama early life career vito corleone ne...,crime drama earli life career vito corleon new...,crime drama early life career vito corleone ne...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Crime, Drama A jury holdout attempts to preven...",crime drama jury holdout attempts prevent misc...,crime drama juri holdout attempt prevent misca...,crime drama jury holdout attempt prevent misca...


In [37]:
idx = 0

content = df.loc[idx, 'metadata']
content

'Drama Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. Frank Darabont Tim Robbins Morgan Freeman Bob Gunton William Sadler'

In [38]:
tf = CountVectorizer(stop_words='english', tokenizer=word_tokenize) 
bow = tf.fit_transform(df['text-prep-stem']) 

In [39]:
idx = 0

content = df.loc[idx, 'text-prep-stem']
watched = tf.transform([content])
dist = cosine_distances(watched, bow)
rec_idx = dist.argsort()[0, 1:11]
df.loc[rec_idx]

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,metadata,text-prep,text-prep-stem,text-prep-lemm
505,https://m.media-amazon.com/images/M/MV5BMTIzND...,Mystic River,2003,A,138 min,"Crime, Drama, Mystery",7.9,The lives of three men who were childhood frie...,84.0,Clint Eastwood,Sean Penn,Tim Robbins,Kevin Bacon,Emmy Rossum,419420,"90,135,191","Crime, Drama, Mystery The lives of three men w...",crime drama mystery lives three men childhood ...,crime drama mysteri live three men childhood f...,crime drama mystery life three men childhood f...
817,https://m.media-amazon.com/images/M/MV5BZWY0OD...,Short Cuts,1993,R,188 min,"Comedy, Drama",7.7,The day-to-day lives of several suburban Los A...,79.0,Robert Altman,Andie MacDowell,Julianne Moore,Tim Robbins,Bruce Davison,42275,"6,110,979","Comedy, Drama The day-to-day lives of several ...",comedy drama daytoday lives several suburban l...,comedi drama daytoday live sever suburban los ...,comedy drama daytoday life several suburban lo...
773,https://m.media-amazon.com/images/M/MV5BMTY5NT...,Brokeback Mountain,2005,A,134 min,"Drama, Romance",7.7,The story of a forbidden and secretive relatio...,87.0,Ang Lee,Jake Gyllenhaal,Heath Ledger,Michelle Williams,Randy Quaid,323103,"83,043,761","Drama, Romance The story of a forbidden and se...",drama romance story forbidden secretive relati...,drama romanc stori forbidden secret relationsh...,drama romance story forbidden secretive relati...
167,https://m.media-amazon.com/images/M/MV5BODM3YW...,Unforgiven,1992,A,130 min,"Drama, Western",8.2,Retired Old West gunslinger William Munny relu...,85.0,Clint Eastwood,Clint Eastwood,Gene Hackman,Morgan Freeman,Richard Harris,375935,"101,157,447","Drama, Western Retired Old West gunslinger Wil...",drama western retired old west gunslinger will...,drama western retir old west gunsling william ...,drama western retired old west gunslinger will...
68,https://m.media-amazon.com/images/M/MV5BMTI3NT...,Oldeuboi,2003,A,101 min,"Action, Drama, Mystery",8.4,After being kidnapped and imprisoned for fifte...,77.0,Chan-wook Park,Choi Min-sik,Yoo Ji-Tae,Kang Hye-jeong,Kim Byeong-Ok,515451,"707,481","Action, Drama, Mystery After being kidnapped a...",action drama mystery kidnapped imprisoned fift...,action drama mysteri kidnap imprison fifteen y...,action drama mystery kidnapped imprisoned fift...
234,https://m.media-amazon.com/images/M/MV5BMTkxNz...,Million Dollar Baby,2004,UA,132 min,"Drama, Sport",8.1,A determined woman works with a hardened boxin...,86.0,Clint Eastwood,Hilary Swank,Clint Eastwood,Morgan Freeman,Jay Baruchel,635975,"100,492,203","Drama, Sport A determined woman works with a h...",drama sport determined woman works hardened bo...,drama sport determin woman work harden box tra...,drama sport determined woman work hardened box...
671,https://m.media-amazon.com/images/M/MV5BMjI5Nj...,Awakenings,1990,U,121 min,"Biography, Drama",7.8,The victims of an encephalitis epidemic many y...,74.0,Penny Marshall,Robert De Niro,Robin Williams,Julie Kavner,Ruth Nelson,125276,"52,096,475","Biography, Drama The victims of an encephaliti...",biography drama victims encephalitis epidemic ...,biographi drama victim enceph epidem mani year...,biography drama victim encephalitis epidemic m...
877,https://m.media-amazon.com/images/M/MV5BODQ0M2...,Dark Waters,2019,PG-13,126 min,"Biography, Drama, History",7.6,A corporate defense attorney takes on an envir...,73.0,Todd Haynes,Mark Ruffalo,Anne Hathaway,Tim Robbins,Bill Pullman,60408,NaN,"Biography, Drama, History A corporate defense ...",biography drama history corporate defense atto...,biographi drama histori corpor defens attorney...,biography drama history corporate defense atto...
311,https://m.media-amazon.com/images/M/MV5BNzc1MT...,The Shop Around the Corner,1940,NaN,99 min,"Comedy, Drama, Romance",8.1,Two employees at a gift shop can barely stand ...,96.0,Ernst Lubitsch,Mar

In [40]:
import pickle

pickle.dump(tf, open('model/tf.pkl', 'wb'))
pickle.dump(bow, open('model/bow.pkl', 'wb'))


In [41]:
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,metadata,text-prep,text-prep-stem,text-prep-lemm
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Drama Two imprisoned men bond over a number of...,drama two imprisoned men bond number years fin...,drama two imprison men bond number year find s...,drama two imprisoned men bond number year find...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Crime, Drama An organized crime dynasty's agin...",crime drama organized crime dynastys aging pat...,crime drama organ crime dynasti age patriarch ...,crime drama organized crime dynasty aging patr...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Action, Crime, Drama When the menace known as ...",action crime drama menace known joker wreaks h...,action crime drama menac known joker wreak hav...,action crime drama menace known joker wreaks h...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Crime, Drama The early life and career of Vito...",crime drama early life career vito corleone ne...,crime drama earli life career vito corleon new...,crime drama early life career vito corleone ne...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Crime, Drama A jury holdout attempts to preven...",crime drama jury holdout attempts prevent misc...,crime drama juri holdout attempt prevent misca...,crime drama jury holdout attempt prevent misca...


In [42]:
master = df[['Poster_Link', 'Series_Title', 'Genre', 'Director', 'text-prep-stem']]
master

,Poster_Link,Series_Title,Genre,Director,text-prep-stem
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,Drama,Frank Darabont,drama two imprison men bond number year find s...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,"Crime, Drama",Francis Ford Coppola,crime drama organ crime dynasti age patriarch ...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,action crime drama menac known joker wreak hav...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,crime drama earli life career vito corleon new...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,"Crime, Drama",Sidney Lumet,crime drama juri holdout attempt prevent misca...
...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,"Comedy, Drama, Romance",Blake Edwards,comedi drama romanc young new york socialit be...
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,"Drama, Western",George Stevens,drama western sprawl epic cover life texa catt...
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,"Drama, Romance, War",Fred Zinnemann,drama romanc war hawaii privat cruelli punish ...
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,"Drama, War",Alfred Hitchcock,drama war sever survivor torpedo merchant ship...


In [43]:
master.to_csv('dataset/master.csv', index=False)